## Aggregate Window Functions
- SUM() 함수의 확장판이라고 봐도 좋습니다. 
- 일반적인 Sum() 함수는 다음과 같습니다. 
    + 2000년대 이후의 전체 매달 갯수를 구하도록 합니다. 

```SQL
WITH Athlete_Medals AS (SELECT
    Country, COUNT(*) AS Medals
FROM Summer_Medals
WHERE Medal = 'Gold' AND Year >= 2000
GROUP BY Country)

SELECT SUM(medals) FROM Athlete_Medals;
 sum
------
 2627
(1개 행)
```

- 이번에는 국가별 Gold 메달 갯수를 알파벳 순서대로 작업을 하도록 합니다. 

```SQL
WITH Athlete_Medals AS (SELECT
    Country, COUNT(*) AS Medals
FROM Summer_Medals
WHERE Medal = 'Gold' AND Year >= 2000
GROUP BY Country)

SELECT SUM(medals) FROM Athlete_Medals;

WITH Athlete_Medals AS (SELECT
    Country, COUNT(*) AS Medals
FROM Summer_Medals
WHERE Medal = 'Gold' AND Year >= 2000
GROUP BY Country)

SELECT 
	Country, 
	Medals, 
	SUM(Medals) OVER (ORDER BY Country ASC) AS Max_Medals
FROM Athlete_Medals
ORDER BY Country ASC;

 country | medals | max_medals
---------+--------+------------
 ALG     |      2 |          2
 ARG     |     47 |         49
 AUS     |    159 |        208
 AUT     |      6 |        214
 AZE     |      6 |        220
 BAH     |     11 |        231
 BEL     |      2 |        233
 BLR     |     16 |        249
 BRA     |     46 |        295
 BUL     |      8 |        303
 CAN     |     19 |        322
 CHI     |      3 |        325
 CHN     |    221 |        546
 CMR     |     20 |        566
 COL     |      2 |        568
 CRO     |     30 |        598
 CUB     |     60 |        658
 CZE     |     10 |        668
 DEN     |     46 |        714
 DOM     |      3 |        717
 EGY     |      1 |        718
 ESP     |     20 |        738
 EST     |      2 |        740
 ETH     |     13 |        753
 FIN     |      4 |        757
 FRA     |     98 |        855
 GBR     |    118 |        973
 GEO     |      6 |        979
 GER     |    159 |       1138
 GRE     |     12 |       1150
 GRN     |      1 |       1151
 HUN     |     77 |       1228
 INA     |      5 |       1233
 IND     |      1 |       1234
 IRI     |     11 |       1245
 IRL     |      1 |       1246
 ISR     |      1 |       1247
 ITA     |     70 |       1317
 JAM     |     23 |       1340
 JPN     |     56 |       1396
 KAZ     |      9 |       1405
 KEN     |     11 |       1416
 KOR     |     85 |       1501
 LAT     |      3 |       1504
 LTU     |      5 |       1509
 MAR     |      2 |       1511
 MEX     |     21 |       1532
 MGL     |      2 |       1534
 MOZ     |      1 |       1535
 NED     |     92 |       1627
 NOR     |     57 |       1684
 NZL     |     18 |       1702
 PAN     |      1 |       1703
 POL     |     20 |       1723
 POR     |      1 |       1724
 PRK     |      6 |       1730
 ROU     |     57 |       1787
 RSA     |     10 |       1797
 RUS     |    203 |       2000
 SLO     |      5 |       2005
 SRB     |      1 |       2006
 SUI     |      7 |       2013
 SVK     |      9 |       2022
 SWE     |     11 |       2033
 THA     |      6 |       2039
 TPE     |      3 |       2042
 TTO     |      1 |       2043
 TUN     |      3 |       2046
 TUR     |      8 |       2054
 UAE     |      1 |       2055
 UGA     |      1 |       2056
 UKR     |     31 |       2087
 USA     |    518 |       2605
 UZB     |      5 |       2610
 VEN     |      1 |       2611
 YUG     |     12 |       2623
 ZIM     |      2 |       2625
         |      2 |       2627
(78개 행)
```

- 이번에는 한국과 일본 국가만 조회될 수 있도록 쿼리를 작성한다. 

```SQL
WITH Country_Medals AS (
  SELECT
    Year, Country, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('KOR', 'JPN')
    AND Medal = 'Gold' AND Year >= 2000
  GROUP BY Year, Country)

SELECT
  Country,
  Year,
  Medals,
  MAX(Medals) OVER (PARTITION BY Country
                        ORDER BY Year ASC) AS Max_Medals
FROM Country_Medals
ORDER BY Country ASC, Year ASC;

 country | year | medals | max_medals
---------+------+--------+------------
 JPN     | 2000 |      5 |          5
 JPN     | 2004 |     21 |         21
 JPN     | 2008 |     23 |         23
 JPN     | 2012 |      7 |         23
 KOR     | 2000 |     12 |         12
 KOR     | 2004 |     14 |         14
 KOR     | 2008 |     41 |         41
 KOR     | 2012 |     18 |         41
(8개 행)

```

```SQL
WITH Country_Awards AS (
    SELECT
      Country,
      Year,
      COUNT(*) AS Awards
    FROM Summer_Medals
    WHERE
      Country IN ('FRA', 'GBR', 'GER')
      AND Year IN (2004, 2008, 2012)
      AND Medal = 'Gold'
    GROUP BY Country, Year)

  SELECT
    Country,
    Year,
    RANK() OVER
      (PARTITION BY Year
       ORDER BY Awards DESC) :: INTEGER AS rank
  FROM Country_Awards
  ORDER BY Country ASC, Year ASC;
  
 country | year | rank
---------+------+------
 FRA     | 2004 |    2
 FRA     | 2008 |    3
 FRA     | 2012 |    3
 GBR     | 2004 |    3
 GBR     | 2008 |    2
 GBR     | 2012 |    1
 GER     | 2004 |    1
 GER     | 2008 |    1
 GER     | 2012 |    2
(9개 행)

```


## Pivot
- Before

```SQL
SELECT 
	Country, Year, COUNT(*) AS Awards
FROM summer_medals
WHERE 
	Country IN ('CHN', 'RUS', 'KOR')
	AND Year IN (2004, 2008, 2012)
	AND Medal = 'Gold' AND Sport = 'Gymnastics'
GROUP BY Country, Year
ORDER BY Country ASC, YEAR ASC;
```
```
 country | year | awards 
---------+------+--------
 CHN     | 2004 |      1
 CHN     | 2008 |     21
 CHN     | 2012 |      9
 KOR     | 2012 |      1
 RUS     | 2004 |      7
 RUS     | 2008 |      7
 RUS     | 2012 |      8
(7 rows)

```

- After Pivot

```SQL
SELECT * FROM CROSSTAB(
	$$ WITH Country_Awards AS (
		SELECT 
			Country, Year, COUNT(*) AS Awards
		FROM summer_medals
		WHERE 
			Country IN ('CHN', 'RUS', 'KOR')
			AND Year IN (2004, 2008, 2012)
			AND Medal = 'Gold' AND Sport = 'Gymnastics'
		GROUP BY Country, Year
		ORDER BY Country ASC, YEAR ASC)
	
		SELECT 
			Country, Year, 
			RANK() OVER
			(PARTITION BY Year ORDER BY Awards DESC) :: INTEGER AS Rank
		FROM Country_Awards
		ORDER BY Country ASC, Year ASC $$) 
AS ct(Country character(6) -- 반드시 기존 컬럼의 데이터 타입과 동일하게 작성해야 함
	  , "2004" INTEGER
	  , "2008" INTEGER
	  , "2012" INTEGER);
```
```
 country | 2004 | 2008 | 2012 
---------+------+------+------
 CHN     |    2 |    1 |    1
 KOR     |    3 |      |     
 RUS     |    1 |    2 |    2
(3 rows)
```